# Step 6
You should now have deployed the SQS queues and the Crawler Lambda service. We can use the code in this notebook to test the service.

## 1. View Bucket Contents
Run the code below to view the contents of your bucket.

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('../support')

// create an S3 object to call
s3 = new AWS.S3()

// configure parameters
params = {
    Bucket: process.env.MY_BUCKET_NAME,
//    Prefix: 'rekognition'
}

s3.listObjects(params, function(err, data) {
  support.printS3Objects(data.Contents)
})
$$.clear()

You should see a few images in the bucket, these were added during step 5 when we called the Rekognition API.

## 2. Post a Message
To test our crawler service we can post a message to the crawler SQS Queue. The code below will post a messge referencing a test web page with some images. The test page can be viewed here: http://ai-as-a-service.s3-website-eu-west-1.amazonaws.com

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('../support')

// create an SQS object to call
sqs = new AWS.SQS()

// construct Queue URL
queueUrl = `https://sqs.${process.env.AWS_REGION}.amazonaws.com/${process.env.AWS_ACCOUNT_ID}/${process.env.MY_CRAWLER_QUEUE}`

// construct message parameters
params = {
  MessageBody: JSON.stringify({action: 'download', msg: {url: 'http://ai-as-a-service.s3-website-eu-west-1.amazonaws.com'}}),
  QueueUrl: queueUrl
}

// send the message
sqs.sendMessage(params, (err, data) => {
    if (err) {
        console.log(`QUEUE ERROR: ${err}`)
    }
    else {
      console.log(`message id: ${data.MessageId}`)        
    }
})
$$.clear()

# 3. Check the Results
If you rerun the code in 1. at the top of this notebook, you should see some additoinal files have been downloaded if our crawler executed successfully. You can confirm this if you look in the AWS S3 console.

If you look at the output you should see that there are a number of downloaded images and also a file `status.json`. The status file is generated by the crawler. Let's check the contents of this file by running the code below:

You will need to replace the `Key` entry in the parameters object with your own key path. You can copy and paste this from the above output.

In [ ]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('../support')

// create an S3 object to call
s3 = new AWS.S3()

// configure parameters
params = {
    Bucket: process.env.MY_BUCKET_NAME,
    Key: 'lob2ic9ac.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/status.json' // replace with your key path to status.json
}

s3.getObject(params, function(err, data) {
  if (err) {
    console.log(`QUEUE ERROR: ${err}`)
  }
  else {
    console.log(JSON.stringify(JSON.parse(data.Body.toString()), null, 2))
  }
})
$$.clear()

The output should show an overall status of `downloaded` with a successful status line for each file.

Congratulations! You have now deployed and triggered the crawler Lambda functions through SQS and collected some files for analysis.

In the next step we will pass these to the analysis function.